<a href="https://colab.research.google.com/github/sviii09/MSAI-AICTE/blob/main/newssentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 textblob plotly twilio
import pandas as pd
from textblob import TextBlob
import plotly.express as px
from bs4 import BeautifulSoup
import requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:00


In [4]:
def scrape_news(keyword="AI", num_articles=5):
    url = f"https://news.google.com/search?q={keyword}&hl=en-US"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for item in soup.find_all('article')[:num_articles]:
        try:
            # Updated selector for title
            title_element = item.find('a', class_='JtKRv')
            title = title_element.text if title_element else "No Title"

            # Updated selector for link
            relative_link = item.find('a', class_='WwrzSb')['href']
            link = f"https://news.google.com{relative_link}"

            articles.append({"title": title, "link": link})
        except (AttributeError, TypeError):
            continue  # Skip articles with missing elements

    return pd.DataFrame(articles)

news_df = scrape_news(keyword="Artificial Intelligence")
print(news_df.head())

                                               title  \
0  AI can spontaneously develop human-like commun...   
1        The Catholic Artificial Intelligence Moment   
2  This Is My Top Artificial Intelligence (AI) Ch...   
3  Prediction: This Artificial Intelligence (AI) ...   
4     10 Free Artificial Intelligence Books For 2025   

                                                link  
0  https://news.google.com./read/CBMitwFBVV95cUxQ...  
1  https://news.google.com./read/CBMihAFBVV95cUxN...  
2  https://news.google.com./read/CBMiiAFBVV95cUxN...  
3  https://news.google.com./read/CBMimAFBVV95cUxQ...  
4  https://news.google.com./read/CBMifEFVX3lxTE1W...  


In [5]:
def analyze_sentiment(df):
    df['sentiment'] = df['title'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['sentiment_label'] = df['sentiment'].apply(
        lambda x: "Positive" if x > 0.1 else "Negative" if x < -0.1 else "Neutral"
    )
    return df

news_df = analyze_sentiment(news_df)

In [6]:
fig = px.bar(news_df,
             x='sentiment_label',
             color='sentiment_label',
             title="News Sentiment Distribution",
             hover_data=['title'])
fig.show()

In [8]:
from twilio.rest import Client

def send_sms_alert(df, threshold=-0.5):
    account_sid = 'YOUR_TWILIO_SID'
    auth_token = 'YOUR_TWILIO_TOKEN'
    client = Client(account_sid, auth_token)

    critical_news = df[df['sentiment'] <= threshold]

    for _, row in critical_news.iterrows():
        message = client.messages.create(
            body=f"⚠️ Negative Alert: {row['title']}",
            from_='+1234567890',  # Your Twilio number
            to='+0987654321'     # Recipient number
        )
        print(f"Alert sent for: {row['title']}")

# Uncomment to enable SMS (requires Twilio account)
# send_sms_alert(news_df)

In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__)

@app.route('/')
def home():
    keyword = request.args.get('keyword', 'AI')
    df = scrape_news(keyword)
    df = analyze_sentiment(df)
    return render_template('dashboard.html',
                         tables=[df.to_html(classes='data')],
                         keyword=keyword)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
